#Script 8

In [13]:
import pandas as pd

# Percorso del file ORIGINALE
input_path = "/content/sample_data/monsters.csv"

# Caricamento dataset
df = pd.read_csv(input_path)

# Controllo rapido
df.head()


,name,size_numeric,type,sub_race,ethics,morale,armor_class,hit_points,hit_dice_count,strength,...,skills.athletics,skills.acrobatics,skills.survival,skills.investigation,skills.nature,skills.intimidation,skills.performance,speed.notes,speed.lightwalking,speed.bur.
0,Aboleth,4,Aberration,Aberration,1,-1,17,135,18,21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Acolyte,3,Humanoid,Humanoid,0,0,10,9,2,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Adult Black Dragon,5,Dragon,Black Dragon,-1,-1,19,195,17,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Adult Blue Dragon,5,Dragon,Blue Dragon,1,-1,19,225,18,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Adult Brass Dragon,5,Dragon,Brass Dragon,-1,1,18,172,15,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#pulizia char indesiderati nei nomi dele colonne
df.columns = (
    df.columns
    .str.strip()
    .str.replace("!", "", regex=False)
)


##Imputazione feature

#### Contesto e obiettivo

Il dataset utilizzato per la costruzione del modello di previsione del Challenge Rating (CR) dei mostri
presenta numerosi valori nulli nelle colonne relative alle *skill*.
In Dungeons & Dragons (5ª edizione), tuttavia, l’assenza di una competenza in una skill **non implica un valore nullo**,
bensì che il valore della skill sia determinato esclusivamente dal **modificatore della caratteristica associata**.

L’obiettivo di questa fase di preprocessing è quindi:
- eliminare valori nulli non informativi,
- ricostruire valori coerenti con le regole del gioco,
- fornire al modello feature numeriche semanticamente corrette e più predittive del CR.


In [15]:
# Colonna di partenza
start_column = "speed.walk"
start_idx = df.columns.get_loc(start_column)

# Sostituisce SOLO i NaN con 0 (non tocca i numeri esistenti)
df.iloc[:, start_idx:] = df.iloc[:, start_idx:].fillna(0)


### Relazione tra skill e caratteristiche

In Dungeons & Dragons ogni skill è associata a una specifica caratteristica (ad esempio:
*Athletics* → *Strength*, *Perception* → *Wisdom*).
Nel dataset le caratteristiche sono rappresentate come **score grezzi** (es. 10, 14, 18, 23)
e non come modificatori.

Il modificatore di una caratteristica viene quindi calcolato secondo la formula ufficiale:

$$
\text{modificatore} = (\text{score} - 10) // 2
$$



## Strategia di imputazione delle skill

Per ogni skill:
- se il valore è diverso da 0, viene lasciato invariato (skill in cui il mostro è competente);
- se il valore è 0, viene sostituito con il **modificatore della caratteristica associata**.

Formalmente:
Formalmente, il valore finale della skill è calcolato come:

$$
\text{skill}_{finale}
=
\text{skill}_{originale}
+
\mathbb{1}(\text{skill}_{originale} = 0)
\cdot \text{modificatore(caratteristica)}
$$



In [16]:
skill_to_ability = {
    "skills.athletics": "strength",

    "skills.acrobatics": "dexterity",
    "skills.stealth": "dexterity",

    "skills.arcana": "intelligence",
    "skills.history": "intelligence",
    "skills.investigation": "intelligence",
    "skills.nature": "intelligence",
    "skills.religion": "intelligence",

    "skills.insight": "wisdom",
    "skills.medicine": "wisdom",
    "skills.perception": "wisdom",
    "skills.survival": "wisdom",

    "skills.deception": "charisma",
    "skills.intimidation": "charisma",
    "skills.performance": "charisma",
    "skills.persuasion": "charisma",
}


In [17]:
for skill_col, ability_col in skill_to_ability.items():
    if skill_col in df.columns and ability_col in df.columns:

        # Calcolo modificatore D&D (floor division)
        modifier = (df[ability_col] - 10) // 2

        # Applica il modificatore SOLO se la skill è 0
        df.loc[df[skill_col] == 0, skill_col] = modifier


In [18]:
# Forza le colonne delle skill a interi (rappresentazione no -01,0)
for col in skill_to_ability.keys():
    if col in df.columns:
        df[col] = df[col].astype(int)

In [19]:
output_path = "/content/sample_data/monsters_skills_dd_corrected_int.csv"

df.to_csv(output_path, index=False)

print(f"File salvato correttamente in: {output_path}")


File salvato correttamente in: /content/sample_data/monsters_skills_dd_corrected_int.csv


In [20]:
# Confronto prima/dopo su alcune colonne
df[[
    "strength",
    "skills.athletics",
    "dexterity",
    "skills.stealth",
    "wisdom",
    "skills.perception"
]].head(10)


,strength,skills.athletics,dexterity,skills.stealth,wisdom,skills.perception
0,21,5,9,-1,15,10
1,10,0,10,0,14,2
2,23,6,14,7,13,11
3,25,7,10,5,15,12
4,23,6,10,5,13,11
5,25,7,10,5,15,12
6,23,6,12,6,15,12
7,27,8,14,8,15,14
8,23,6,12,6,15,12
9,27,8,10,6,13,13


### Motivazioni delle scelte

Questa strategia è stata adottata perché:
- rispetta le regole ufficiali di Dungeons & Dragons;
- evita di introdurre valori arbitrari o puramente statistici;
- riduce il rumore dovuto a valori nulli;
- preserva la coerenza semantica tra caratteristiche e skill;
- migliora la qualità delle feature in ingresso al modello di machine learning.

Di conseguenza, il modello di previsione del CR riceve informazioni più realistiche e strutturate,
migliorando la sua capacità di apprendere la pericolosità effettiva dei mostri.